# Reed-Solomon Codes
Authors: Alex Matsoukas, Dakota Chang, Lila Smith

Date last edited: 05 May 2024

Description: Final project for `The Theory and Application of Algebraic Structures` at Olin College of Engineering, Spring 2024. This notebook contains an explanation of Reed-Solomon Codes and includes a basic implementation using the Python library [reedsolo](https://github.com/tomerfiliba-org/reedsolomon).

---
Reed-Solomon codes are a subset of BCH codes, introduced in 1960 in a paper by Reed and Solomon. Their resistance to burst errors has led to their use in both CDs and deep space communications where scratches or cosmic radiation could interrupt consecutive bits.

Reed-Solomon (RS) Codes are a type of cyclic code.
A *cyclic code* is a code with the following properties:

Given codewords $c_1$ and $c_2$ inside a cyclic code $C$:
- Any linear combination of $c_1$ and $c_2$ is in $C$
- Any cyclic shift of $c_1$ is in $C$ (ex: 010110 -> 101100)

If we encode any codeword $c = c_0c_1c_2c_3...c_{n-1}$ in the code $C$ as a polynomial $c(x) = c_0 + c_1x + c_2x^2 + c_3x^3 + ... + c_{n-1}x^{n-1}$, then we can treat cyclic codes as ideals in the ring $GF(q)[x] / x^n - 1$. For Reed-Solomon codes, groups of bits, called digits or symbols, are encoded rather than single bits.  

What does this mean?
- First, we can define a *field* as a set with a two-group structure. All field elements form an additive commutative group, and all non-zero field elements form a multiplicative commutative group.
- A field of finite order is called a *Galois Field*.
- $GF(q)$ is a Galois Field of order $q$, where $q$ is always a power of a prime.
- $GF(q)[x]$ is the ring of polynomials with coefficients from $GF(q)$.
- $GF(q)[x] / x^n - 1$ means that the polynomials are modded by $x^n - 1$. In other words, $x^n - 1$ wraps around to 0. In math terms, $x^n - 1 = 0$, which means that $x^n = 1$. This implies that the highest order possible for a polynomial in $GF(q)[x] / x^n - 1$ is $n - 1$.

- An *ideal* is a subring of a ring with the special property that any operation between an element in the ring and an element in the ideal results in an element inside the ideal.

With these definitions, we can define cyclic codes in a new way:

Let $c(x) = c_0 + c_1x + c_2x^2 + ... + c_{n-1}x^{n-1}$ be a member of $GF(q)[x] / x^n - 1$. Multiplying $c(x)$ by $x$ gives:

$xc(x) = c_0x + c_1x^2 + c_2x^3 + ... + c_{n-2}x^{n-1} + c_{n-1}x^n$. But since we are modding by $x^n - 1$, this is actually:

$xc(x) = c_{n-1} + c_0x + c_1x^2 + c_2x^3 + ... + c_{n-2}x^{n-1}$

In codeword form, $c(x)$ would be $c_0c_1c_2...c_{n-1}$ and $xc(x)$ would be $c_{n-1}c_0c_1c_2...c_{n-2}$.

This means that within the ring $GF(q)[x] / x^n - 1$, multiplying by $x$ is a shift of the polynomial (or codeword) to the right, and multiplying by subsequent powers of $x$ would be additional shifts. Furthermore, addition of polynomials in the ring is the same as taking linear combinations of codewords. So, an ideal in the ring $GF(q)[x] / x^n - 1$ would be one in which all shifts of a polynomial in the ideal stay in the ideal and all linear combinations of a polynomial in the ideal with another polynomial stay in the ideal - this is the definition of a cyclic code.

So, making an RS Code is a problem of finding an ideal in $GF(q)[x] / x^n - 1$. To find them, we need to be able to factor $x^n - 1$ because generators of ideals will divide $x^n - 1$. 

In any finite field $GF(q)$, there exists a *primitive element*, $\alpha$, such that the order of $\alpha$ is $q-1$; furthermore, all powers of $\alpha$ are unique, which means that every non-zero element in $GF(q)$ can be generated by $\alpha$. Additionally, if we pick our $n$ to be $q-1$, then each of the powers of $\alpha$ are now zeros of $x^n - 1 = x^{q-1} - 1$, by definition of order. RS Codes capable of correcting $t$ errors are created by taking $2t$ consecutive powers of $\alpha$. 

As you can see, with RS Codes, the length of the codeword must be tied to the size of the field.

## Key Functions Utilized in this Notebook: A Brief Overview

### `rs_generator_poly(nsym, fcr=0, generator=2)`

This function generates an irreducible generator polynomial over a finite field. In the context of fields and rings:

- **Field**: A field is a set equipped with two operations, usually addition and multiplication, where addition and multiplication are commutative, associative, and distributive over each other, and every nonzero element has a multiplicative inverse. In Reed-Solomon coding, we work with finite fields, denoted as GF(q), where "q" is a power of a prime.
- **Irreducible Polynomial**: An irreducible polynomial over a field is a polynomial that cannot be factored into the product of two non-constant polynomials over the same field. Irreducible polynomials are fundamental in creating finite fields and constructing algebraic codes like Reed-Solomon codes.

### `rs_generator_poly_all(max_nsym, fcr=0, generator=2)`

This function generates all irreducible generator polynomials up to a specified maximum number of symbols (nsym) over a finite field. It iterates through each possible number of symbols and constructs the corresponding irreducible generator polynomial using the `rs_generator_poly` function.

- **Finite Field**: A finite field, denoted as GF(q), is a field with a finite number of elements. It is constructed using an irreducible polynomial. In the context of Reed-Solomon coding, the finite field is used for arithmetic operations on symbols.
- **Generator Polynomial**: In Reed-Solomon coding, the generator polynomial defines the structure of codewords. It is constructed using irreducible polynomials over the finite field.

### `init_tables(prim=0x11d, generator=2, c_exp=8)`

This function initializes logarithm and anti-logarithm tables for efficient arithmetic operations within the finite field.

- **Logarithm and Anti-logarithm Tables**: Given any base or generator $b$, the expression $ b^{(\text{log}_b(x), \text{log}_b(y))} = x \times y$ holds true. This means that exponentiating the logarithms of two elements in a finite field with the same base results in the product of those elements. By precomputing logarithm and anti-logarithm tables using a chosen base or generator, efficient multiplication and division operations can be achieved through table lookups instead of costly exponentiation calculations.

In [10]:
# my laptop has weird things with paths, ignore this chunk
import sys
sys.path.append("/opt/homebrew/lib/python3.11/site-packages")

from tabulate import tabulate
import reedsolo as rs
import random
import string

In [27]:
temp_c_exp = 3
fast_primes = rs.find_prime_polys(c_exp=temp_c_exp, fast_primes=True, single=True)
rs_init_codes = rs.init_tables(c_exp=temp_c_exp, prim=fast_primes)

# Splitting byte arrays and joining the items with comma
bytearray1_items = ','.join([str(byte) for byte in rs_init_codes[0]])
bytearray2_items = ','.join([str(byte) for byte in rs_init_codes[1]])

# Convert items to polynomial representation
bytearray1_items_poly = ""
bytearray2_items_poly  = ""

for i in range(len(rs_init_codes[0])):
    bytearray1_items_poly += (str(rs_init_codes[0][i]) + "x^" + str(i) + " + ")

bytearray1_items_poly = bytearray1_items_poly[:-3]
    
for i in range(len(rs_init_codes[1])):
    bytearray2_items_poly += (str(rs_init_codes[1][i]) + "x^" + str(i) + " + ")

bytearray2_items_poly = bytearray2_items_poly[:-3]

# Creating tables
table1 = [["Byte Array 1", bytearray1_items, bytearray1_items_poly]]
table2 = [["Byte Array 2", bytearray2_items, bytearray2_items_poly]]

# Printing tables
print(tabulate(table1, headers=["Item", "Representation", "Polynomial Representation"]))
print("\n")
print(tabulate(table2, headers=["Item", "Representation", "Polynomial Representation"]))

Item          Representation    Polynomial Representation
------------  ----------------  -----------------------------------------------------
Byte Array 1  0,0,1,3,2,6,4,5   0x^0 + 0x^1 + 1x^2 + 3x^3 + 2x^4 + 6x^5 + 4x^6 + 5x^7


Item          Representation               Polynomial Representation
------------  ---------------------------  ---------------------------------------------------------------------------------------------------
Byte Array 2  1,2,4,3,6,7,5,1,2,4,3,6,7,5  1x^0 + 2x^1 + 4x^2 + 3x^3 + 6x^4 + 7x^5 + 5x^6 + 1x^7 + 2x^8 + 4x^9 + 3x^10 + 6x^11 + 7x^12 + 5x^13


### Prime Polynomials 

Because we are using a binary code (not, for instance, ternary), the order of our field (number of elements) will be a power of 2, based on the length of our message. For instance, if we want to have 16 bits of information, our field would be $GF(2^{4})$. In this field, we need to find an irreducible polynomial, aka a polynomial that cannot be factored in this field.

It is also worth noting that our polynomial coefficients will be from {0,1}, aka $GF(2)$, because it is a binary code. As such, we can represent the polynomial $g(x) = x^2 + 1$ as 0101. This polynomial can be noted to be reducible as $x^2 + 1 = (x+1)^2$ in $GF(2^4)$.

We will use this polynomial to provide conversions for $\alpha$ where $\alpha$ is an element of $GF(n)$ and has an order of $n-1$. Since it has this order, it is a generator of the field since it can form $n-1$ unique elements. 


The fast primes algorithm will find fewer prime polynomials but will do so faster. Since we only *need* one prime polynomial for our lookup table, we can use this option. Without this option, the algorithm will check every possible polynomial (that isn't divisible by 2). For most size common sized fields, lookup tables exist that give these polynomials.

In [32]:
prim = rs.find_prime_polys(c_exp=12, fast_primes=False, single=True)
string_repr = str(bin(prim))[2:]
poly = " + ".join(filter(None, [(char=="1") * f"x^{len(string_repr) - 1 - i}" \
                                for i, char in enumerate(string_repr)]))
print(f"One irreducible polynomial in GF(2^4) is {prim:b}, aka {poly}.")

One irreducible polynomial in GF(2^4) is 1000001010011, aka x^12 + x^6 + x^4 + x^1 + x^0.


[array('i', [0, 0, 1, 2159, 2, 223, 2160, 1448, 3, 3607, 224, 500, 2161, 4011, 1449, 2382, 4, 446, 3608, 3125, 225, 2896, 501, 2075, 2162, 3727, 4012, 1671, 1450, 2659, 2383, 3935, 5, 1999, 447, 1364, 3609, 3084, 3126, 723, 226, 2142, 2897, 1791, 502, 3830, 2076, 679, 2163, 1948, 3728, 2605, 4013, 1189, 1672, 538, 1451, 139, 2660, 3984, 2384, 571, 3936, 960, 6, 3119, 2000, 896, 448, 1000, 1365, 2730, 3610, 1804, 3085, 2298, 3127, 2048, 724, 1080, 227, 3927, 2143, 12, 2898, 669, 1792, 4000, 503, 2697, 3831, 1319, 2077, 1288, 680, 3348, 2164, 1745, 1949, 63, 3729, 3523, 2606, 1599, 4014, 2882, 1190, 249, 1673, 2706, 539, 1148, 1452, 2838, 140, 2580, 2661, 1752, 3985, 1894, 2385, 478, 572, 206, 3937, 3950, 961, 416, 7, 2575, 3120, 2600, 2001, 1986, 897, 2014, 449, 1298, 1001, 2637, 1366, 2365, 2731, 3178, 3611, 1024, 1805, 902, 3086, 644, 2299, 3396, 3128, 4053, 2049, 3851, 725, 3377, 1081, 3911, 228, 1337, 3928, 3904, 2144, 2222, 13, 158, 2899, 3758, 670, 23, 1793, 3625, 4001, 1587, 504,

### Sending a Message

In [79]:
n = 255  # length of total message+ecc
nsym = 12  # length of ecc

letters = string.ascii_lowercase  # lowercase letters
mes = ''.join(random.choice(letters) for _ in range(n - nsym))

In [99]:
print("Our message:", mes)

Our message: hoeivsynjwtjzobbfxyogvxommatqrzzlqlntjrowmnptrcpvearmvyhvcifeqdzaytkovnbszpbxvnpyrejsjaulexcgyxsmhwqfppdowwuaiuzjuiijsgytstibgisqhehbmncruykcwalryygydiaozokyohfeblbqecueztdumhnwrjqiymjsngigmnwflxrgndaoaimgaosdlqccgqmnmxzuipeueeyauslhgeqwpjyodj


In [100]:
# Generator polynomial
gen = rs.rs_generator_poly_all(n)
print(gen)

{0: array('i', [1]), 1: array('i', [1, 1]), 2: array('i', [1, 3, 2]), 3: array('i', [1, 7, 14, 8]), 4: array('i', [1, 15, 54, 120, 64]), 5: array('i', [1, 31, 198, 792, 1984, 1024]), 6: array('i', [1, 63, 806, 2955, 1322, 3873, 664]), 7: array('i', [1, 127, 3302, 479, 3832, 477, 1628, 1086]), 8: array('i', [1, 255, 915, 3477, 522, 1959, 2990, 1480, 1331]), 9: array('i', [1, 511, 4018, 782, 3396, 1991, 1363, 3968, 3908, 598]), 10: array('i', [1, 1023, 3523, 1566, 4068, 3078, 2862, 2296, 4030, 3332, 2733]), 11: array('i', [1, 2047, 7, 2044, 3428, 2569, 1372, 2846, 1954, 2001, 2124, 277]), 12: array('i', [1, 4095, 4066, 3849, 2770, 155, 1651, 3860, 1990, 2342, 800, 739, 792]), 13: array('i', [1, 4012, 208, 1676, 1775, 3117, 2056, 2144, 3404, 2852, 2590, 3746, 6, 393]), 14: array('i', [1, 3850, 3147, 2005, 2959, 3346, 2664, 3078, 431, 105, 257, 2828, 3655, 605, 951]), 15: array('i', [1, 3654, 3998, 3972, 1216, 4051, 2021, 3928, 1533, 2441, 276, 572, 2924, 1527, 2956, 1976]), 16: array('i',

In [105]:
mesecc = rs.rs_encode_msg(mes, nsym, gen=gen[nsym])

### Add errors

In [106]:
# Generating a list of 6 random indexes within the range of the list 'mesecc'.

random_list_indexes = [random.randint(0,len(mesecc)) for _ in range(6)]

print("These parts of the message, ", end="")

for i in random_list_indexes:
    print("mesecc[" + str(i) + "]: " + str(mesecc[i]), end=", ")
    mesecc[i] = 0

print("are all converted into 0 to replicate a series of errors.", end="")

These parts of the message, mesecc[237]: 112, mesecc[199]: 97, mesecc[77]: 118, mesecc[60]: 101, mesecc[242]: 106, mesecc[49]: 101, are all converted into 0 to replicate a series of errors.

Correcting errors in the message using a function `rs.rs_correct_msg()`, with parameters `mesecc` and `nsym`, then checking the corrected message.

In [107]:
rmes, recc, errata_pos = rs.rs_correct_msg(mesecc, nsym)
rs.rs_check(rmes + recc, nsym)
text_rmes = ""
for letter in rmes:
    text_rmes += chr(letter)
print("Original Message:", mes)
print("Received Message:", text_rmes)

Original Message: hoeivsynjwtjzobbfxyogvxommatqrzzlqlntjrowmnptrcpvearmvyhvcifeqdzaytkovnbszpbxvnpyrejsjaulexcgyxsmhwqfppdowwuaiuzjuiijsgytstibgisqhehbmncruykcwalryygydiaozokyohfeblbqecueztdumhnwrjqiymjsngigmnwflxrgndaoaimgaosdlqccgqmnmxzuipeueeyauslhgeqwpjyodj
Received Message: hoeivsynjwtjzobbfxyogvxommatqrzzlqlntjrowmnptrcpvearmvyhvcifeqdzaytkovnbszpbxvnpyrejsjaulexcgyxsmhwqfppdowwuaiuzjuiijsgytstibgisqhehbmncruykcwalryygydiaozokyohfeblbqecueztdumhnwrjqiymjsngigmnwflxrgndaoaimgaosdlqccgqmnmxzuipeueeyauslhgeqwpjyodj


In [108]:
print('Number of detected errors and erasures: %i, their positions: %s' % (len(errata_pos), list(errata_pos)))

Number of detected errors and erasures: 6, their positions: [242, 237, 199, 77, 60, 49]


We have a ratio of 243 data bits to 12 parity bits and are able to correct 